In [115]:
import sys
from PIL import Image
from math import log10, sqrt
import cv2
import numpy as np
import math
import imageio

# A

In [41]:
def immirror(img) :
    width  = img.size[0]
    height = img.size[1]
    for y in range(height):
        for x in range(width//2):
            left = img.getpixel((x, y))
            right = img.getpixel((width - 1 - x, y))
            img.putpixel((width - 1 - x, y), left)
            img.putpixel((x, y), right)
    return img

In [116]:
img = Image.open(r'erling_haaland.png')
immirror(img)
img.save('img_mirror.png')

# B

In [111]:
img = imageio.imread("zlatan_ibrahimovic.png")
height, width, channel = img.shape
width_cutoff = width // 2
R = img[:, :width_cutoff]
L = img[:, width_cutoff:]
imageio.imwrite("cristiano_ronaldor.png", R)
imageio.imwrite("cristiano_ronaldol.png", L)

In [117]:
image1 = Image.open("cristiano_ronaldor.png")
immirror(image1)
image1.save("cristiano_ronaldor_i.png")
image2 = Image.open("cristiano_ronaldol.png")
immirror(image2)
image2.save("cristiano_ronaldol_i.png")

# C

In [108]:
def get_concat_h_blank(im1, im2, color=(0, 0, 0)):
    dst = Image.new('RGB', (im1.width + im2.width, max(im1.height, im2.height)), color)
    dst.paste(im1, (0, 0))
    dst.paste(im2, (im1.width, 0))
    return dst



im1 = Image.open("cristiano_ronaldor.png")
im2 = Image.open("cristiano_ronaldor_i.png")
get_concat_h_blank(im1, im2).save("cristiano_ronaldo1_2.png")

im3 = Image.open("cristiano_ronaldol.png")
im4 = Image.open("cristiano_ronaldol_i.png")
get_concat_h_blank(im4, im3).save("cristiano_ronaldo2_2.png")


get_concat_h_blank(im4, im2).save("cristiano_ronaldo3_2.png")   
    

In [118]:
def impsnr(img, ref):
    mse = np.mean((img - ref) ** 2)
    if(mse == 0): 
        return 100
    max_pixel = 255.0
    psnr = 20 * log10(max_pixel / sqrt(mse))
    return psnr

def main():
    original = cv2.imread("original_image.png")
    compressed = cv2.imread("compressed_image.png", 1)
    value = PSNR(original, compressed)
    print(f"PSNR value is {value} dB")

if __name__ == "__main__":
    main()


PSNR value is 0.6845571030618441 dB


In [72]:
def impsnr(img, ref):
    img1 = img.astype(np.float64)
    img2 = ref.astype(np.float64)
    mse = np.mean((img - ref)**2)
    if mse == 0:
        return float('inf')
    return 20 * math.log10(255.0 / math.sqrt(mse))


In [113]:
print('zlatan_ibrahimovic:')
ref = cv2.imread("cristiano_ronaldo.png")
img = cv2.imread("cristiano_ronaldo1_2.png")
resized_image = cv2.resize(img, (533, 605)) 
value = impsnr(resized_image, ref)
print(f"PSNR value  for Right, Right_inverse  is {value} dB")

img = cv2.imread("cristiano_ronaldo2_2.png")
resized_image = cv2.resize(img, (533, 605)) 
value = impsnr(resized_image, ref)
print(f"PSNR value  for left, left_inverse is {value} dB")

img = cv2.imread("cristiano_ronaldo3_2.png")
resized_image = cv2.resize(img, (533, 605)) 
value = impsnr(resized_image, ref)
print(f"PSNR value  for Right_inverse, Left_inverse is {value} dB")

zlatan_ibrahimovic:
PSNR value  for Right, Right_inverse  is 31.515931261656817 dB
PSNR value  for left, left_inverse is 31.431193755917064 dB
PSNR value  for Right_inverse, Left_inverse is 31.53356364843933 dB


In [67]:
def ssim(img1, img2):
    C1 = (0.01 * 255)**2
    C2 = (0.03 * 255)**2
    img1 = img1.astype(np.float64)
    img2 = img2.astype(np.float64)
    kernel = cv2.getGaussianKernel(11, 1.5)
    window = np.outer(kernel, kernel.transpose())
    mu1 = cv2.filter2D(img1, -1, window)[5:-5, 5:-5]  # valid
    mu2 = cv2.filter2D(img2, -1, window)[5:-5, 5:-5]
    mu1_sq = mu1**2
    mu2_sq = mu2**2
    mu1_mu2 = mu1 * mu2
    sigma1_sq = cv2.filter2D(img1**2, -1, window)[5:-5, 5:-5] - mu1_sq
    sigma2_sq = cv2.filter2D(img2**2, -1, window)[5:-5, 5:-5] - mu2_sq
    sigma12 = cv2.filter2D(img1 * img2, -1, window)[5:-5, 5:-5] - mu1_mu2
    ssim_map = ((2 * mu1_mu2 + C1) * (2 * sigma12 + C2)) / ((mu1_sq + mu2_sq + C1) *
                                                            (sigma1_sq + sigma2_sq + C2))
    return ssim_map.mean()


def  imssim(img, ref):
    if not img.shape == ref.shape:
        raise ValueError('Input images must have the same dimensions.')
    if img.ndim == 2:
        return ssim(img, ref)
    elif img.ndim == 3:
        if img.shape[2] == 3:
            ssims = []
            for i in range(3):
                ssims.append(ssim(img, ref))
            return np.array(ssims).mean()
        elif img.shape[2] == 1:
            return ssim(np.squeeze(img), np.squeeze(ref))
    else:
        raise ValueError('Wrong input image dimensions.')


In [114]:
print('zlatan_ibrahimovic:')
ref = cv2.imread("cristiano_ronaldo.png")
img = cv2.imread("cristiano_ronaldo1_2.png")
resized_image = cv2.resize(img, (533, 605)) 
value = imssim(resized_image, ref)
print(f"immsim value  for Right, Right_inverse  is {value} dB")

img = cv2.imread("cristiano_ronaldo2_2.png")
resized_image = cv2.resize(img, (533, 605)) 
value = imssim(resized_image, ref)
print(f"immsim value  for Left, Left_inverse is {value} dB")

img = cv2.imread("cristiano_ronaldo3_2.png")
resized_image = cv2.resize(img, (533, 605)) 
value = imssim(resized_image, ref)
print(f"immsim value  for Right_inverse, Left_inverse is {value}")

zlatan_ibrahimovic:
immsim value  for Right, Right_inverse  is 0.6888398758672264 dB
immsim value  for Left, Left_inverse is 0.6825249353588041 dB
immsim value  for Right_inverse, Left_inverse is 0.6845571030618441
